In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install datasets
! pip install -U accelerate
! pip install -U transformers

In [ ]:
def generate_combinations_permutations(lst ):

    r = len( lst )

    all_permutations = []

    for i in range(r) :

        combinations = itertools.combinations(lst, i+1)

        for combination in combinations:
            permutations = list(itertools.permutations(combination))
            all_permutations.extend(permutations)

    return all_permutations

In [ ]:
import itertools

temp = [  "訂位", "詢問", "導航", "推薦" ]



all_possible = generate_combinations_permutations( temp )

print( len(all_possible) )

print( all_possible )
print( len(all_possible[0]) )

In [ ]:
def Label_For_Sequence( all_possible ) :

    all_possible_dict = {}
    all_possible_dict[ ('聊天',) ] = 0
    number = 1
    last = 1

    for i in all_possible:

        # print( "now" + str(len(i)) )
        # print( "last" + str(last) )

        if len(i) == 1 :
            all_possible_dict[ i ] = 0
        elif len(i) != last :
            number = 1
            all_possible_dict[ i ] = bin( number )
            number = number + 1
        else:
            all_possible_dict[ i ] = bin( number )
            number = number + 1

        last = len(i)


    print( all_possible_dict )



    return all_possible_dict



all_possible_dict = Label_For_Sequence( all_possible )

In [ ]:
def Get_Binary_Address( binary ) :

    address = []
    binary = str(binary)
    reverse_binary = binary[::-1]

    for number, i in enumerate( reverse_binary ):
        if i == "1" :
            address.append( number + 1 )
        elif i == "b":
            break


    return address



In [ ]:
import pandas as pd

def Read( filename ) :

    with open( f"/content/drive/MyDrive/分類/功能/{ filename }.txt", 'r', encoding='utf-8' ) as f :
        data = f.readlines()


    new_data = {}
    label = []
    g_times = 0
    text = ""

    for i in data :

        if "<G>" in i :

            context = i.split( "<G>" )
            i = context[1]

            g_times = g_times + 1
            if g_times == 2 :
                new_data[text] = label
                label = []
                g_times = 0
                text = ""
        elif "<USER>" in i :

            context = i.split( "<USER>" )
            i = context[1]

            if text == "" :
                text =  i.strip()
            else :
                text = text + "\n" + i.strip()
        elif "<LABEL>" in i :

            context = i.split( "<LABEL>" )
            i = context[1]

            label.append( i.strip() )
        elif "<BOT>" in i :

            context = i.split( "<BOT>" )
            i = context[1]


            if text == "" :
                text =  i.strip()
            else :
                text = text + "\n" + i.strip()



    # print(new_data)

    return new_data

In [ ]:
def Create_Label_Dictonary( label_dict, binary,  ) :

    copy_data = data.copy()

In [ ]:
classes =  [ '聊天', '導航', '推薦', '訂位', '詢問', 1, 2, 3, 4, 5]

answer = [0, 0, 0, 0, 0, 0, 0, 0, 0 ,0]

label_dict = {}

for i in all_possible_dict:
  for k in i:
    answer[ classes.index( k ) ] = 1

  address = Get_Binary_Address( all_possible_dict[i] )

  for k in address:
    answer[ classes.index( k ) ] = 1

  print( i )
  print( answer )
  print("===================================")

  label_dict[ tuple( answer ) ] = i



  answer = [0, 0, 0, 0, 0, 0, 0, 0, 0 ,0]

print( label_dict )


In [ ]:
list(('訂位',))

In [ ]:
read = Read( "詢問+導航/data1" )


read = Input_Label( read, all_possible_dict )

read

In [ ]:
def  Input_Label( data, all_possible_dict ) :

    copy_data = data.copy()

    for i in copy_data :
        # print( i )
        if tuple( copy_data[i] ) in all_possible_dict :
            address = Get_Binary_Address( all_possible_dict[ tuple( copy_data[i] ) ] )
            # print( address )

            # print("=======================")
            for k in address :
                # print(k)
                copy_data[i].append( str(k) )


    # print( copy_data )
    return copy_data



# print(data)

In [ ]:
def Change_Fomat_Data( data ) :
    all_data = {}
    all_datas = {}
    texts = []
    labels = []
    for text in data :
        texts.append( text )
        labels.append( data[ text ] )

    all_data[ 'text' ] = texts
    all_data[ 'label' ] = labels

    # all_datas[ name ] = all_data

    return all_data


In [ ]:
def Counter_Label( data ):

    count_dict = {}
    for key in data :
      value = data[key]
      sentence = ""
      for text in value :
        sentence = sentence + text

      # print(value)
      # print(key)
      # print(sentence)
      if sentence not in count_dict :
          count_dict[ sentence ] = 1
      else :
          count_dict[ sentence ] = count_dict[ sentence ] + 1

    print( count_dict )

Counter_Label( data )

In [ ]:
#為了將類別進行公平分類
def Fair_Split_Data( data ) :

  label = []
  all_data = {}
  for i in data :
    if data[i] not in label :
      label.append( data[i] )
      all_data[ str( data[i] ) ] = []
      all_data[ str( data[i] ) ].append( i )
    else :
      all_data[ str( data[i] ) ].append( i )

  return all_data


In [ ]:
import ast

#改變資料的型態 從dict變成 text, label 的兩個list
def Change_Fomat_Data_Fair( data ) :
    all_data = {}

    for label in data:
      texts = []
      labels = []
      for text in data[label]:
          texts.append(text)
          labels.append( ast.literal_eval(label) )

      all_data[f'{label}_text'] = texts
      all_data[f'{label}_label'] = labels


    return all_data

In [ ]:


#把 text, label 的兩個list 的各類別分配到 Dataset.from_dict
def Datasetify( datas, names ) :

  data_function = {}

  if len(names) != len(datas)/2 :
    print( len(names) )
    print( len(datas) )
    print("ERROR")
  else:
    return_list = []

    store = 0

    for number, data in enumerate( datas ) :


      store = store + 1

      if store == 1 :
        data_function['text'] = datas[ data ]
      elif store == 2 :
        data_function['label'] = datas[ data ]
        store = 0

      if store == 0 :
        ds_dict[ names[ int( number/2 ) ]  ] = Dataset.from_dict( data_function )


        data_function = {}

    return ds_dict

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
from datasets import concatenate_datasets

filenames = [ '聊天/data1', '推薦/single_data1_for_onelabels', '導航/single_data1_for_onelabels', '訂位/single_data1_for_onelabels',
'詢問/single_data1_for_onelabels', '推薦+導航/data1', '推薦+導航/data2', '推薦+訂位/data1', '推薦+訂位/data2', '推薦+詢問/data1', '推薦+詢問/data2',
 '訂位+詢問/data1','訂位+詢問/data2','詢問+導航/data1','詢問+導航/data2','訂位+導航/data1','訂位+導航/data2']


all_data = {}
ds_dict = DatasetDict()

data = {}
# print( title )
for number, filename in enumerate( filenames ) :

    read = Read( filename )


    data.update( Input_Label( read, all_possible_dict ) )




print( data )
all_data = Fair_Split_Data( data )

print( all_data )

dataset = Change_Fomat_Data_Fair( all_data )

print( dataset )




# data = Read( '推薦\\single_data1_for_onelabels' )

# len( all_data )


ds_dict = Datasetify( dataset, ['chat', 'recommand', 'map', 'reserve', 'ask', 'recommand_map', 'map_recommand', 'recommand_reserve', 'reserve_recommand', 'recommand_ask', 'ask_recommand',
                      'reserve_ask', 'ask_reserve', 'ask_map', 'map_ask', 'reserve_map', 'map_reserve'])

ds_dict


In [ ]:
def Split( ds_dict, train_size, classification ) :

    train_test =  ds_dict[ classification ].train_test_split( train_size=train_size, shuffle=True )

    print( classification )
    print( train_test )

    return train_test


train_test1 = Split( ds_dict, 0.6 ,'recommand' )
train_test2 = Split( ds_dict, 0.8 ,'reserve' )
train_test3 = Split( ds_dict, 0.8 ,'map' )
train_test4 = Split( ds_dict, 0.8 , 'ask' )


train_test5 = Split( ds_dict, 0.9 ,'recommand_map' )
train_test6 = Split( ds_dict, 0.9 ,'map_recommand' )
train_test7 = Split( ds_dict, 0.9 ,'recommand_reserve' )
train_test8 = Split( ds_dict, 0.9 ,'reserve_recommand' )
train_test9 = Split( ds_dict, 0.9 ,'recommand_ask' )
train_test10 = Split( ds_dict, 0.9 ,'ask_recommand' )
train_test11 = Split( ds_dict, 0.9 ,'reserve_ask' )
train_test12 = Split( ds_dict, 0.9 ,'ask_reserve' )
train_test13 = Split( ds_dict, 0.9 ,'ask_map' )
train_test14 = Split( ds_dict, 0.9 ,'map_ask' )
train_test15 = Split( ds_dict, 0.9 ,'reserve_map' )
train_test16 = Split( ds_dict, 0.9 ,'map_reserve' )

train_test17 = Split( ds_dict, 0.5 ,'chat' )




# print( train_test1 )

In [ ]:

df1 = pd.DataFrame(train_test1['train'])
df2 = pd.DataFrame(train_test2['train'])
df3 = pd.DataFrame(train_test3['train'])

df4 = pd.DataFrame(train_test4['train'])
df5 = pd.DataFrame(train_test5['train'])

df6 = pd.DataFrame(train_test6['train'])
df7 = pd.DataFrame(train_test7['train'])

df8 = pd.DataFrame(train_test8['train'])
df9 = pd.DataFrame(train_test9['train'])
df10 = pd.DataFrame(train_test10['train'])

df11 = pd.DataFrame(train_test11['train'])
df12 = pd.DataFrame(train_test12['train'])

df13 = pd.DataFrame(train_test13['train'])
df14 = pd.DataFrame(train_test14['train'])
df15 = pd.DataFrame(train_test15['train'])
df16 = pd.DataFrame(train_test16['train'])
df17 = pd.DataFrame(train_test17['train'])

df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17], axis=0, ignore_index=True )  # 合併

from sklearn.preprocessing import MultiLabelBinarizer

classes = ['聊天', '導航', '推薦', '訂位', '詢問', '1', '2', '3', '4', '5']
mlb = MultiLabelBinarizer(classes=classes)


labels = mlb.fit_transform(df['label']).astype( "float32" )  # 把label轉化成數字 對應 classes 有:1 無: 0   mlb.fit_transform() 格示[[a],[b]....]

df_final = pd.concat( [df['text'], pd.DataFrame(labels,columns=list(mlb.classes_))],axis=1 ) # 將文字 和 pandas( label ) 組合


text = df_final['text'].tolist()  # 取出文字 變成list

print( text )

In [ ]:
df_final

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,DistilBertForTokenClassification, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")


import torch
from torch.utils.data import Dataset

class C_Dataset( Dataset ) :
  def __init__( self, label, text ) :
    self.label = label
    self.text = text

  def __len__( self ):
    return len( self.text )

  def __getitem__( self, idx ) :
    text = str( self.text[idx] )
    label = torch.tensor( self.label[idx] )

    encoding = tokenizer( text,  max_length=200, truncation=True, padding="max_length", return_tensors='pt' )

    # print( encoding )

    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'labels': label
    }

In [ ]:
train_dataset = C_Dataset( labels, text )

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,DistilBertForTokenClassification, DistilBertForSequenceClassification
print('labels_len:', len(labels[0]) )
tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
model = AutoModelForSequenceClassification.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student", num_labels=len(labels[0]), ignore_mismatched_sizes=True)

In [ ]:
from transformers import  Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/BERT_classification_multi_class/new_result',
    num_train_epochs = 300,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,
    learning_rate=5e-5,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
    save_steps=1000,
    max_steps=3000,
    run_name='my_experiment_1',
    report_to='none'
    # evaluation_strategy = 'epoch',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # compute_metrics=compute_metrics,
    # eval_dataset=test_dataset
)

In [ ]:
trainer.train()

## **TEST**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification ,DistilBertForTokenClassification, DistilBertForSequenceClassification
# print('labels_len:', len(labels[0]) )
tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/BERT_classification_multi_class/new_result/checkpoint-3000")

In [ ]:
df1 = pd.DataFrame(train_test1['test'])
df2 = pd.DataFrame(train_test2['test'])
df3 = pd.DataFrame(train_test3['test'])

df4 = pd.DataFrame(train_test4['test'])
df5 = pd.DataFrame(train_test5['test'])

df6 = pd.DataFrame(train_test6['test'])
df7 = pd.DataFrame(train_test7['test'])

df8 = pd.DataFrame(train_test8['test'])
df9 = pd.DataFrame(train_test9['test'])
df10 = pd.DataFrame(train_test10['test'])

df11 = pd.DataFrame(train_test11['test'])
df12 = pd.DataFrame(train_test12['test'])

df13 = pd.DataFrame(train_test13['test'])
df14 = pd.DataFrame(train_test14['test'])
df15 = pd.DataFrame(train_test15['test'])
df16 = pd.DataFrame(train_test16['test'])
df17 = pd.DataFrame(train_test17['test'])

df_test = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16, df17], axis=0, ignore_index=True )  # 合併

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import torch


classes = ['聊天','導航', '推薦', '訂位', '詢問', '1', '2', '3', '4', '5']
mlb = MultiLabelBinarizer(classes=classes)



labels = mlb.fit_transform(df_test['label'])  # 把label轉化成數字 對應 classes 有:1 無: 0   mlb.fit_transform() 格示[[a],[b]....]

labels = labels.tolist()

print(labels)
print(df_test['label'])


text = df_test['text'].tolist()


def Predict( text, label ) :

  answer = [0, 0, 0, 0, 0, 0, 0, 0, 0 ,0]

  encoding = tokenizer( text, return_tensors='pt')
  encoding.to( model.device )

  outputs = model( **encoding )


  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid( outputs.logits[0].cpu() )

  print( probs )
  print( label )

  print("SSSSSSSSSSSSSSSSSSSSSSSSSS")

  positions = ( probs > 0.6 ).nonzero( as_tuple=True )[0].tolist()


  for i in positions:
    answer[i] = 1

  if answer == label :
    return 1
  else :
    return 0



def Predict_answer( text ) :

  answer = [0, 0, 0, 0, 0, 0, 0, 0, 0 ,0]

  encoding = tokenizer( text, return_tensors='pt')
  encoding.to( model.device )

  outputs = model( **encoding )


  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid( outputs.logits[0].cpu() )


  positions = ( probs > 0.6 ).nonzero( as_tuple=True )[0].tolist()

  for i in positions:
    answer[i] = 1

  return answer


def Test_answer( text ) :

  pre_label = []

  correct = 0

  for la, i in enumerate( text ):

    predict = Predict( i, labels[la] )

    if predict == 1 :
      correct = correct + 1

    pre_label.append( Predict_answer( i ) )

  print(correct)
  print(correct/len(text))

  return pre_label

pre_label = Test_answer( text )

# print( Predict_answer( "你好" ) )


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tabulate import tabulate


labels = labels # 標籤為字符串

# 模型預測結果（這裡是假設的預測）
predictions = pre_label

accuracy = accuracy_score(labels, predictions)


report = classification_report(labels, predictions, output_dict=True)

f1_macro_avg = report['macro avg']['f1-score']
f1_weighted_avg = report['weighted avg']['f1-score']


# 填充到字典中
data = {
    'Model': ['Bart'],  # 這裡填寫你的模型名稱，例如： 'My Model'
    'Accuracy': [accuracy],
    'F1 Score': [f1_weighted_avg]  # 使用加權平均的 F1 分數
}

# 這樣可以將結果印出或轉換為 DataFrame 以便展示
import pandas as pd
df = pd.DataFrame(data)

table = tabulate(df, headers='keys', tablefmt='grid')  # 可更改 'grid' 为 'latex' 或其他格式

print(table)